In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf_device = '/gpu:2'

In [16]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from uncertainty_from_distance import uncertainty_from_distances
import matplotlib.pyplot as plt
from sampling_unc import sample_uniform
from attack import attack_fn
from sampling_unc import sample_high_unc, sample_low_unc

In [17]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(100)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(100)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [18]:
fc_model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
fc_model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

fc_model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6


2022-06-24 16:11:25.059750: E tensorflow/stream_executor/cuda/cuda_blas.cc:232] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-06-24 16:11:25.059787: E tensorflow/stream_executor/cuda/cuda_blas.cc:234] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2022-06-24 16:11:25.059804: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at matmul_op_impl.h:438 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support


InternalError: Graph execution error:

Detected at node 'sequential_2/dense_4/MatMul' defined at (most recent call last):
    File "/home/interns2022/miniconda3/envs/sarthak/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/interns2022/miniconda3/envs/sarthak/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/interns2022/miniconda3/envs/sarthak/lib/python3.10/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/interns2022/miniconda3/envs/sarthak/lib/python3.10/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/interns2022/miniconda3/envs/sarthak/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/home/interns2022/miniconda3/envs/sarthak/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 

In [ ]:
fc_distance_obj = uncertainty_from_distances(fc_model, 100, 20, 28, 1)

In [ ]:
ds_fc_uni_sample = sample_uniform(ds_train, 0.1, fc_distance_obj, 28, 1)

In [13]:
ds_fc_uni_sample = tf.slice(ds_fc_uni_sample, [0, 0, 0, 0], [tf.shape(ds_fc_uni_sample)[0] - 1, 28, 28, 1]) 
y = tf.random.uniform([len(ds_fc_uni_sample)], minval = 0, maxval = 9)
ds_fc_uniform_sampled = tf.data.Dataset.from_tensor_slices((ds_fc_uni_sample, y))
ds_fc_uniform_sampled = ds_fc_uniform_sampled.cache()
ds_fc_uniform_sampled = ds_fc_uniform_sampled.shuffle(ds_info.splits['train'].num_examples)
ds_fc_uniform_sampled = ds_fc_uniform_sampled.batch(100)
ds_fc_uniform_sampled = ds_fc_uniform_sampled.prefetch(tf.data.AUTOTUNE)

NameError: name 'ds_fc_uni_sample' is not defined

In [14]:
ds_fc_high_sample = sample_high_unc(ds_train, 0.1, fc_distance_obj, 28, 1)

2022-06-24 16:10:36.593860: E tensorflow/stream_executor/cuda/cuda_blas.cc:232] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-06-24 16:10:36.593898: E tensorflow/stream_executor/cuda/cuda_blas.cc:234] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2022-06-24 16:10:36.593922: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at matmul_op_impl.h:438 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support


InternalError: Exception encountered when calling layer "dense" (type Dense).

Attempting to perform BLAS operation using StreamExecutor without BLAS support [Op:MatMul]

Call arguments received by layer "dense" (type Dense):
  • inputs=tf.Tensor(shape=(100, 784), dtype=float32)

In [10]:
ds_fc_high_sample = tf.slice(ds_fc_high_sample, [0, 0, 0, 0], [tf.shape(ds_fc_high_sample)[0] - 1, 28, 28, 1]) 
y = tf.random.uniform([len(ds_fc_high_sample)], minval = 0, maxval = 9)
ds_fc_high_sampled = tf.data.Dataset.from_tensor_slices((ds_fc_high_sample, y))
ds_fc_high_sampled = ds_fc_high_sampled.cache()
ds_fc_high_sampled = ds_fc_high_sampled.shuffle(ds_info.splits['train'].num_examples)
ds_fc_high_sampled = ds_fc_high_sampled.batch(100)
ds_fc_high_sampled = ds_fc_high_sampled.prefetch(tf.data.AUTOTUNE)

In [11]:
ds_rand_uniform = tf.random.uniform([60000, 28, 28, 1], minval = 0, maxval = 1)
y = tf.random.uniform([60000], minval = 0, maxval = 9)
ds_rand_uniform_sampled = tf.data.Dataset.from_tensor_slices((ds_rand_uniform, y))
ds_rand_uniform_sampled = ds_rand_uniform_sampled.cache()
ds_rand_uniform_sampled = ds_rand_uniform_sampled.shuffle(ds_info.splits['train'].num_examples)
ds_rand_uniform_sampled = ds_rand_uniform_sampled.batch(100)
ds_rand_uniform_sampled = ds_rand_uniform_sampled.prefetch(tf.data.AUTOTUNE)

In [ ]:
ds_fc_low_sample = sample_low_unc(ds_train, 0.1, fc_distance_obj, 28, 1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
ds_fc_low_sample = tf.slice(ds_fc_low_sample, [0, 0, 0, 0], [tf.shape(ds_fc_low_sample)[0] - 1, 28, 28, 1]) 
y = tf.random.uniform([len(ds_fc_low_sample)], minval = 0, maxval = 9)
ds_fc_low_sampled = tf.data.Dataset.from_tensor_slices((ds_fc_low_sample, y))
ds_fc_low_sampled = ds_fc_low_sampled.cache()
ds_fc_low_sampled = ds_fc_low_sampled.shuffle(ds_info.splits['train'].num_examples)
ds_fc_low_sampled = ds_fc_low_sampled.batch(100)
ds_fc_low_sampled = ds_fc_low_sampled.prefetch(tf.data.AUTOTUNE)

In [ ]:
stolen_1 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_2 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_3 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_4 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_5 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_6 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_7 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_8 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_9 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_10 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_11 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_12 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])

In [ ]:
attack_fn(fc_model, stolen_1, ds_train, 10, 28, 1, 100, False, ds_test)

In [ ]:
attack_fn(fc_model, stolen_2, ds_train, 10, 28, 1, 100, False, ds_test)

In [ ]:
attack_fn(fc_model, stolen_3, ds_fc_uniform_sampled, 10, 28, 1, 100, False, ds_test)

In [ ]:
attack_fn(fc_model, stolen_4, ds_fc_uniform_sampled, 10, 28, 1, 100, False, ds_test)

In [ ]:
attack_fn(fc_model, stolen_5, ds_fc_high_sampled, 10, 28, 1, 100, False, ds_test)

In [ ]:
attack_fn(fc_model, stolen_6, ds_fc_high_sampled, 10, 28, 1, 100, False, ds_test)

In [ ]:
attack_fn(fc_model, stolen_7, ds_rand_uniform_sampled, 10, 28, 1, 100, False, ds_test)

In [ ]:
attack_fn(fc_model, stolen_8, ds_rand_uniform_sampled, 10, 28, 1, 100, False, ds_test)

In [ ]:
attack_fn(fc_model, stolen_9, ds_rand_uniform_sampled, 10, 28, 1, 100, True, ds_test)

In [ ]:
attack_fn(fc_model, stolen_10, ds_rand_uniform_sampled, 10, 28, 1, 100, True, ds_test)

In [ ]:
attack_fn(fc_model, stolen_11, ds_fc_low_sampled, 10, 28, 1, 100, False, ds_test)

In [ ]:
attack_fn(fc_model, stolen_12, ds_fc_low_sampled, 10, 28, 1, 100, False, ds_test)

In [ ]:
cnn_model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(10, (3, 3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.Flatten(input_shape=(26, 26, 10)),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
cnn_model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

cnn_model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

In [ ]:
cnn_distance_obj = uncertainty_from_distances(cnn_model, 100, 20, 28, 1)

In [ ]:
ds_cnn_uni_sample = sample_uniform(ds_train, 0.1, cnn_distance_obj)

In [ ]:
ds_cnn_uni_sample = tf.slice(ds_cnn_uni_sample, [0, 0, 0, 0], [tf.shape(ds_cnn_uni_sample)[0] - 1, 28, 28, 1]) 
y = tf.random.uniform([len(ds_cnn_uni_sample)], minval = 0, maxval = 9)
ds_cnn_uniform_sampled = tf.data.Dataset.from_tensor_slices((ds_cnn_uniform_sampled, y))
ds_cnn_uniform_sampled = ds_cnn_uniform_sampled.cache()
ds_cnn_uniform_sampled = ds_cnn_uniform_sampled.shuffle(ds_info.splits['train'].num_examples)
ds_cnn_uniform_sampled = ds_cnn_uniform_sampled.batch(100)
ds_cnn_uniform_sampled = ds_cnn_uniform_sampled.prefetch(tf.data.AUTOTUNE)

In [ ]:
ds_cnn_high_sample = sample_high_unc(ds_train, 0.1, cnn_distance_obj, 28, 1)

In [ ]:
ds_cnn_high_sample = tf.slice(ds_cnn_high_sample, [0, 0, 0, 0], [tf.shape(ds_cnn_high_sample)[0] - 1, 28, 28, 1]) 
y = tf.random.uniform([len(ds_cnn_high_sample)], minval = 0, maxval = 9)
ds_cnn_high_sampled = tf.data.Dataset.from_tensor_slices((ds_cnn_high_sample, y))
ds_cnn_high_sampled = ds_cnn_high_sampled.cache()
ds_cnn_high_sampled = ds_cnn_high_sampled.shuffle(ds_info.splits['train'].num_examples)
ds_cnn_high_sampled = ds_cnn_high_sampled.batch(100)
ds_cnn_high_sampled = ds_cnn_high_sampled.prefetch(tf.data.AUTOTUNE)

In [ ]:
ds_cnn_low_sample = sample_low_unc(ds_train, 0.1, cnn_distance_obj, 28, 1)

In [ ]:
ds_cnn_low_sample = tf.slice(ds_cnn_low_sample, [0, 0, 0, 0], [tf.shape(ds_cnn_low_sample)[0] - 1, 28, 28, 1]) 
y = tf.random.uniform([len(ds_cnn_low_sample)], minval = 0, maxval = 9)
ds_cnn_low_sampled = tf.data.Dataset.from_tensor_slices((ds_cnn_low_sample, y))
ds_cnn_low_sampled = ds_cnn_low_sampled.cache()
ds_cnn_low_sampled = ds_cnn_low_sampled.shuffle(ds_info.splits['train'].num_examples)
ds_cnn_low_sampled = ds_cnn_low_sampled.batch(100)
ds_cnn_low_sampled = ds_cnn_low_sampled.prefetch(tf.data.AUTOTUNE)

In [ ]:
stolen_13 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_14 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_15 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_16 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_17 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_18 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_19 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_20 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_21 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_22 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_23 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])
stolen_24 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax(axis = -1)
])

In [ ]:
attack_fn(fc_model, stolen_13, ds_train, 10, 28, 1, 100, False, ds_test)

In [ ]:
###########change from fc yo cnn in model everywhere here

In [ ]:
attack_fn(fc_model, stolen_14, ds_train, 10, 28, 1, 100, False, ds_test)

In [ ]:
attack_fn(fc_model, stolen_15, ds_cnn_uniform_sampled, 10, 28, 1, 100, False, ds_test)

In [ ]:
attack_fn(fc_model, stolen_16, ds_cnn_uniform_sampled, 10, 28, 1, 100, False, ds_test)

In [ ]:
attack_fn(fc_model, stolen_17, ds_cnn_high_sampled, 10, 28, 1, 100, False, ds_test)

In [ ]:
attack_fn(fc_model, stolen_18, ds_cnn_high_sampled, 10, 28, 1, 100, False, ds_test)

In [ ]:
attack_fn(fc_model, stolen_19, ds_rand_uniform_sampled, 10, 28, 1, 100, False, ds_test)

In [ ]:
attack_fn(fc_model, stolen_20, ds_rand_uniform_sampled, 10, 28, 1, 100, False, ds_test)

In [ ]:
attack_fn(fc_model, stolen_21, ds_rand_uniform_sampled, 10, 28, 1, 100, True, ds_test)

In [ ]:
attack_fn(fc_model, stolen_22, ds_cnn_low_sampled, 10, 28, 1, 100, True, ds_test)

In [ ]:
attack_fn(fc_model, stolen_22, ds_cnn_low_sampled, 10, 28, 1, 100, True, ds_test)

In [7]:
label_list_1 = np.array([0])

for (im, lab) in ds_fc_uniform_sampled:
    waste, labels = fc_distance_obj.get_top1_class(im)
    label_list_1 = np.append(label_list_1, tf.make_ndarray(tf.make_tensor_proto(labels)), axis = 0)
    
plt.hist(label_list_1, bins = 10)
plt.show()

NameError: name 'ds_fc_high_sampled' is not defined

In [ ]:
label_list_2 = np.array([0])

for (im, lab) in ds_fc_high_sampled:
    waste, labels = fc_distance_obj.get_top1_class(im)
    label_list_2 = np.append(label_list_2, tf.make_ndarray(tf.make_tensor_proto(labels)), axis = 0)
    
plt.hist(label_list_2, bins = 10)
plt.show()

In [ ]:
label_list_3 = np.array([0])

for (im, lab) in ds_fc_low_sampled:
    waste, labels = fc_distance_obj.get_top1_class(im)
    label_list_3 = np.append(label_list_3, tf.make_ndarray(tf.make_tensor_proto(labels)), axis = 0)
    
plt.hist(label_list_3, bins = 10)
plt.show()

In [ ]:
label_list_4 = np.array([0])

for (im, lab) in ds_cnn_uniform_sampled:
    waste, labels = cnn_distance_obj.get_top1_class(im)
    label_list_4 = np.append(label_list_4, tf.make_ndarray(tf.make_tensor_proto(labels)), axis = 0)
    
plt.hist(label_list_4, bins = 10)
plt.show()

In [ ]:
label_list_5 = np.array([0])

for (im, lab) in ds_cnn_high_sampled:
    waste, labels = cnn_distance_obj.get_top1_class(im)
    label_list_5 = np.append(label_list_5, tf.make_ndarray(tf.make_tensor_proto(labels)), axis = 0)
    
plt.hist(label_list_5, bins = 10)
plt.show()

In [ ]:
label_list_6 = np.array([0])

for (im, lab) in ds_cnn_low_sampled:
    waste, labels = cnn_distance_obj.get_top1_class(im)
    label_list_6 = np.append(label_list_6, tf.make_ndarray(tf.make_tensor_proto(labels)), axis = 0)
    
plt.hist(label_list_6, bins = 10)
plt.show()